# Create a Project

In [1]:
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags

user_id = UserId.new()
project_id = ProjectId.new()
name = Name.from_str('Project 1')
description = Description.from_str('Description 1')
tags = Tags.from_list(['tag1', 'tag2'])

from morpheus.project.application.write.ProjectCommandHandlers import CreateProjectCommand, CreateProjectCommandHandler

create_project_command = CreateProjectCommand.new(
  project_id=project_id,
  name=name,
  description=description,
  tags=tags,
  user_id=user_id
)

CreateProjectCommandHandler.handle(create_project_command)

# Read ProjectList

In [2]:
from morpheus.project.application.read.ProjectsReader import ProjectsReader

projects_reader = ProjectsReader()
project_list = projects_reader.get_project_summaries()
project_list

[ProjectSummary(project_id=ProjectId(value='8f6660d6-2a2b-406c-afe1-5bcf25085de7'), project_name=Name(value='s'), project_description=Description(value='s'), project_tags=Tags(value=[]), owner_id=UserId(value='1ce566df-10d1-4f4b-b9c5-71af60b1896a'), visibility=<Visibility.PRIVATE: 'private'>, created_at=DateTime(value=datetime.datetime(2024, 3, 26, 20, 23, 3, 143296)), updated_at=DateTime(value=datetime.datetime(2024, 3, 26, 20, 23, 3, 143296))),
 ProjectSummary(project_id=ProjectId(value='48a04932-b4e4-4864-a74f-7f390eac2193'), project_name=Name(value='s'), project_description=Description(value='s'), project_tags=Tags(value=[]), owner_id=UserId(value='1ce566df-10d1-4f4b-b9c5-71af60b1896a'), visibility=<Visibility.PRIVATE: 'private'>, created_at=DateTime(value=datetime.datetime(2024, 3, 27, 11, 33, 36, 607694)), updated_at=DateTime(value=datetime.datetime(2024, 3, 27, 11, 33, 36, 607694))),
 ProjectSummary(project_id=ProjectId(value='a6db379c-b297-4f49-bc7e-a3373cf7c3b4'), project_name

# Create model

In [3]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit
from morpheus.project.types.geometry import Polygon
from morpheus.project.types.Model import ModelId
from morpheus.project.application.write.ModelCommandHandlers import CreateModelCommand, CreateModelCommandHandler

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

n_cols = 100
n_rows = 50
rotation = 0.0

create_model_command = CreateModelCommand.new(
  project_id=project_id,
  user_id=user_id,
  model_id=ModelId.new(),
  geometry=polygon,
  n_cols=n_cols,
  n_rows=n_rows,
  rotation=rotation,
)

CreateModelCommandHandler.handle(create_model_command)

# Read Latest Model

In [4]:
from morpheus.project.application.read.ModelReader import ModelReader

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
model

Model(model_id=ModelId(value='144b785b-eeab-4b6b-8218-93e549547210'), spatial_discretization=SpatialDiscretization(geometry=Polygon(coordinates=[[[13.922514437551428, 50.964720483303836], [13.925250781947113, 50.965228748412386], [13.925036413951403, 50.96623732041704], [13.92222441026388, 50.96629040370362], [13.922514437551428, 50.964720483303836]]], type='Polygon'), grid=Grid(origin=Point(coordinates=[13.922224410263878, 50.9662904037036], type='Point'), col_widths=[3.3689415472885593, 3.3689415472885593, 3.3689415472885598, 3.368941547288559, 3.368941547288559, 3.3689415472885607, 3.3689415472885607, 3.368941547288557, 3.368941547288557, 3.3689415472885607, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.368941547288564, 3.36894154728855, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.3689415472885713, 3.368941547288557, 3.368941547288557, 3.368941547288557, 3.368941547288557, 3.3689415472885713, 3.3689

## Update Model Geometry


In [5]:
from morpheus.project.application.write.ModelCommandHandlers import UpdateModelGeometryCommandHandler, UpdateModelGeometryCommand

new_polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922, 50.965),
    (13.925, 50.965),
    (13.925, 50.966),
    (13.922, 50.966),
    (13.922, 50.965)
  ]]
)

command = UpdateModelGeometryCommand.new(
  project_id=project_id,
  geometry=new_polygon,
  user_id=user_id
)

UpdateModelGeometryCommandHandler.handle(command)

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
model.spatial_discretization.geometry

Polygon(coordinates=[[[13.922, 50.965], [13.925, 50.965], [13.925, 50.966], [13.922, 50.966], [13.922, 50.965]]], type='Polygon')

## Update Model Grid Properties

In [6]:
from morpheus.project.application.write.ModelCommandHandlers import UpdateModelGridCommandHandler, UpdateModelGridCommand

new_del_col_relative = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
new_del_row_relative = [0, 0.2, 0.4, 0.6, 0.8, 1]

command = UpdateModelGridCommand.new(
  project_id=project_id,
  user_id=user_id,
  n_cols=len(new_del_col_relative),
  n_rows=len(new_del_row_relative),
  rotation=10,
)

UpdateModelGridCommandHandler.handle(command=command)

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
print(model.spatial_discretization.geometry)
print(model.spatial_discretization.grid)
print(model.spatial_discretization.affected_cells.to_dict())

Polygon(coordinates=[[[13.922, 50.965], [13.925, 50.965], [13.925, 50.966], [13.922, 50.966], [13.922, 50.965]]], type='Polygon')
Grid(origin=Point(coordinates=[13.921728464165456, 50.96596984662515], type='Point'), col_widths=[32.68894794320857, 32.68894794320857, 32.68894794320856, 32.688947943208575, 32.6889479432086, 32.68894794320852, 32.6889479432086, 32.68894794320855, 32.68894794320863, 32.688947943208575, 32.68894794320852], total_width=359.57842737529427, row_heights=[38.67716080928221, 38.67716080928221, 38.67716080928221, 38.67716080928221, 38.677160809282185, 38.67716080928224], total_height=232.06296485569328, rotation=Rotation(value=10), length_unit=LengthUnit(unit=2))
{'type': 'raster', 'empty_value': False, 'shape': [6, 11], 'data': [[False, True, True, True, False, False, False, False, False, False, False], [False, True, True, True, True, True, True, True, True, True, True], [True, True, True, True, True, True, True, True, True, True, True], [True, True, True, True, T

## Update Model Affected Cells

In [7]:
from morpheus.project.application.write.ModelCommandHandlers import UpdateModelAffectedCellsCommandHandler, UpdateModelAffectedCellsCommand

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id=project_id)
affected_cells = model.spatial_discretization.affected_cells

assert affected_cells.is_active(col=0, row=0) == False
affected_cells.set_active(col=0, row=0)
assert affected_cells.is_active(col=0, row=0) == True

command = UpdateModelAffectedCellsCommand.new(
  project_id=project_id,
  affected_cells=affected_cells,
  user_id=user_id
)

UpdateModelAffectedCellsCommandHandler.handle(command=command)

In [8]:
model_reader = ModelReader()
model = model_reader.get_latest_model(project_id=project_id)
affected_cells = model.spatial_discretization.affected_cells
assert affected_cells.is_active(col=0, row=0) == True

## Show Versions of Model so far

In [9]:
model_reader = ModelReader()
model_reader.get_versions(project_id=project_id)

[ModelVersion(version_id=VersionId(value='05193852-c73d-4680-92c3-a03cde8938a5'), tag=VersionTag(value='v0.0.0'), description=VersionDescription(value='Initial version'))]

## Tag Version

In [10]:
from morpheus.project.application.write.ModelCommandHandlers import CreateVersionCommandHandler
from morpheus.project.application.write.ModelCommands import CreateVersionCommand
from morpheus.project.types.ModelVersion import VersionTag, VersionDescription

command = CreateVersionCommand.new(
  project_id=project_id,
  version_tag=VersionTag('v1.0.0'),
  version_description=VersionDescription('Version 1.0.0'),
  user_id=user_id
)

CreateVersionCommandHandler.handle(command=command)

In [11]:
model_reader.get_versions(project_id=project_id)

[ModelVersion(version_id=VersionId(value='b766b3fc-2c16-4011-8392-702c5bf75907'), tag=VersionTag(value='v1.0.0'), description=VersionDescription(value='Version 1.0.0')),
 ModelVersion(version_id=VersionId(value='05193852-c73d-4680-92c3-a03cde8938a5'), tag=VersionTag(value='v0.0.0'), description=VersionDescription(value='Initial version'))]

In [12]:
version = model_reader.get_version_by_tag(project_id=project_id, tag=VersionTag('v1.0.0'))
version

ModelVersion(version_id=VersionId(value='b766b3fc-2c16-4011-8392-702c5bf75907'), tag=VersionTag(value='v1.0.0'), description=VersionDescription(value='Version 1.0.0'))

In [13]:
from morpheus.project.application.write.ModelCommandHandlers import UpdateVersionDescriptionCommand, UpdateVersionDescriptionCommandHandler

command = UpdateVersionDescriptionCommand.new(
  project_id=project_id,
  version_id=version.version_id,
  version_description=VersionDescription('Version 1.0.0 - Updated'),
  user_id=user_id
)

UpdateVersionDescriptionCommandHandler.handle(command=command)

In [14]:
version = model_reader.get_version_by_tag(project_id=project_id, tag=VersionTag('v1.0.0'))
version.description.to_str()

'Version 1.0.0 - Updated'